In [36]:
import time
import numpy as np
import pandas as pd
import os
import sys
import sklearn
import datetime
import random
import matplotlib.pyplot as plt
import math
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Activation, Flatten, BatchNormalization, GlobalAveragePooling2D  
from tensorflow.keras.backend import batch_normalization
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import metrics
from packaging import version
%matplotlib inline

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [37]:
#identify GPU
device_name = tf.test.gpu_device_name()
if not tf.test.is_gpu_available():
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [38]:
print("TensorFlow version: ", tf.__version__)

TensorFlow version:  2.1.0


In [51]:
#train data
X_train = np.load(os.path.join("Data_new3", "X_train.npy"))
y_train = np.load(os.path.join("Data_new3", "train_labels_multi.npy"))
y_train_bi = np.load(os.path.join("Data_new3", "y_train.npy"))

#test data
X_test = np.load(os.path.join("Data_new3", "X_test.npy"))
y_test = np.load(os.path.join("Data_new3", "y_test_labels_multi.npy"))
y_test_bi = np.load(os.path.join("Data_new3", "y_test.npy"))

#validation data
X_val = np.load(os.path.join("Data_new3", "X_val.npy"))
y_val = np.load(os.path.join("Data_new3", "y_val_labels_multi.npy"))
y_val_bi = np.load(os.path.join("Data_new3", "y_val.npy"))

In [52]:
#train data
print("X_train data:", X_train.shape)
print("y_train data:", y_train.shape)

X_train data: (8599, 256, 256, 1)
y_train data: (8599,)


In [53]:
print(y_train.shape)

(8599,)


In [54]:
#validation data
print("X_validation data:", X_val.shape)
print("y_validation data:", y_val.shape)

X_validation data: (967, 256, 256, 1)
y_validation data: (967,)


In [55]:
#test data
print("X_test data:", X_test.shape)
print("y_test data:", y_test.shape)

X_test data: (1934, 256, 256, 1)
y_test data: (967,)


In [56]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

#binary
y_train_bi = to_categorical(y_train_bi)
y_val_bi = to_categorical(y_val_bi)
y_test_bi = to_categorical(y_test_bi)

In [57]:
# scale pixels
X_train = X_train/255.0
X_val = X_val/255.0
X_test = X_test/255.0

In [58]:
y_val.shape

(967, 4)

In [59]:
classes = 4
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(256, 256, 1)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(classes, activation='softmax'))
    # compile model
    opt = Adam(lr=0.001, beta_1=0.9, beta_2 = 0.999)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [60]:
model = define_model()

In [61]:
# fit model
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_val, y_val))

Train on 8599 samples, validate on 967 samples
Epoch 1/5
8599/8599 [==============================] - 16s 2ms/sample - loss: 1612.5016 - accuracy: 0.3091 - val_loss: 1.3877 - val_accuracy: 0.3433
Epoch 2/5
8599/8599 [==============================] - 16s 2ms/sample - loss: 1.2801 - accuracy: 0.4004 - val_loss: 1.4184 - val_accuracy: 0.3464
Epoch 3/5
8599/8599 [==============================] - 16s 2ms/sample - loss: 1.2157 - accuracy: 0.4290 - val_loss: 1.5116 - val_accuracy: 0.3485
Epoch 4/5
8599/8599 [==============================] - 15s 2ms/sample - loss: 1.1832 - accuracy: 0.4403 - val_loss: 1.6146 - val_accuracy: 0.3537
Epoch 5/5
8599/8599 [==============================] - 16s 2ms/sample - loss: 1.1690 - accuracy: 0.4462 - val_loss: 1.7622 - val_accuracy: 0.3495


In [63]:
_, acc = model.evaluate(X_test, y_test, verbose=0)
print('> %.3f' % (acc * 100.0))

ValueError: Input arrays should have the same number of samples as target arrays. Found 1934 input samples and 967 target samples.